In [1]:
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
files = ['train-v2.0.json', 'dev-v2.0.json']

In [2]:
import os

squad_dir = 'squad'

if not os.path.exists(squad_dir):
    os.mkdir(squad_dir)

In [3]:
import requests

for file in files:
    res = requests.get(url+file)
    # write to file in chunks
    with open(os.path.join(squad_dir, file), 'wb') as f:
        for chunk in res.iter_content(chunk_size=40):
            f.write(chunk)

In [4]:
import json

with open(os.path.join(squad_dir, 'train-v2.0.json'), 'rb') as f:
    squad = json.load(f)

In [5]:
new_squad = []



for group in squad['data']:
    for paragraph in group['paragraphs']:
        context =paragraph['context']
        for qa_pair in paragraph['qas']:
            question=qa_pair['question']
            if 'answers' in qa_pair.keys() and len(qa_pair['answers'])>0:
                answer=qa_pair['answers'][0]['text']
            elif 'plausible_answers' in qa_pair.keys() and len(qa_pair['plausible_answers']) >0:
                answer = qa_pair['plausible_answers'][0]['text']
            else:
                answer=None
            new_squad.append({
                'question':question,
                'answer':answer,
                'context':context
            })


In [6]:
new_squad[:2], new_squad[-2:]

([{'question': 'When did Beyonce start becoming popular?',
   'answer': 'in the late 1990s',
   'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'},
  {'question': 'What areas did Beyonce compete in when she was growing up?',
   'answer': 'singing and dancing',
   'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born

In [7]:
with open(os.path.join(squad_dir, 'train.json'), 'w') as f:
    json.dump(new_squad, f)

In [8]:
with open(os.path.join(squad_dir,'dev.json'),'rb') as f:
    squad=json.load(f)

In [9]:
new_squad = []



for group in squad['data']:
    for paragraph in group['paragraphs']:
        context =paragraph['context']
        for qa_pair in paragraph['qas']:
            question=qa_pair['question']
            if 'answers' in qa_pair.keys() and len(qa_pair['answers'])>0:
                answer=qa_pair['answers'][0]['text']
            elif 'plausible_answers' in qa_pair.keys() and len(qa_pair['plausible_answers']) >0:
                answer = qa_pair['plausible_answers'][0]['text']
            else:
                answer=None
            new_squad.append({
                'question':question,
                'answer':answer,
                'context':context
            })


In [10]:

import json
with open(os.path.join(squad_dir,'dev1.json'),'w') as f:
    json.dump(new_squad,f)

In [11]:
import json 
with open(r'C:\Users\siddh\Desktop\pythonudmy\nlp\squad\dev1.json','r') as f :
    squad = json.load(f)

In [12]:
from transformers import BertTokenizer, BertForQuestionAnswering
modelname='deepset/bert-base-cased-squad2'
tokenizer=BertTokenizer.from_pretrained(modelname)
model = BertForQuestionAnswering.from_pretrained(modelname)

In [13]:
from transformers import pipeline
qa = pipeline('question-answering',model=model,tokenizer=tokenizer)

In [14]:
answers=[]
for pair in squad[:5]:
    ans=qa({
        'question':pair['question'],
        'context': pair['answer']
    })
    answers.append({
        'predicted':ans['answer'],
        'true':pair['answer']
    })

In [21]:
answers

[{'predicted': 'France', 'true': 'France'},
 {'predicted': '10th and 11th centuries', 'true': '10th and 11th centuries'},
 {'predicted': 'Denmark, Iceland and Norway',
  'true': 'Denmark, Iceland and Norway'},
 {'predicted': 'Rollo', 'true': 'Rollo'},
 {'predicted': '10th', 'true': '10th century'}]

In [37]:
from rouge import Rouge
rouge=Rouge()

In [38]:
model_out = [ans['predicted'] for ans in answers]
reference = [ans['true'] for ans in answers]

In [39]:
rouge.get_scores(model_out,reference,avg=True)

{'rouge-1': {'r': 0.9, 'p': 1.0, 'f': 0.9333333284444445},
 'rouge-2': {'r': 0.4, 'p': 0.4, 'f': 0.399999998},
 'rouge-l': {'r': 0.9, 'p': 1.0, 'f': 0.9333333284444445}}

In [40]:
from tqdm import tqdm

In [41]:
model_out=[]
reference=[]
for pair in tqdm(squad, leave=True):
    ans=qa({
        'question':pair['question'],
        'context':pair['context']
    })
    model_out.append(ans['answer'])
    reference.append(pair['answer'])

  0%|          | 6/11873 [00:36<20:02:22,  6.08s/it]


KeyboardInterrupt: 

In [ ]:
rouge.get_scores(model_out,reference,avg=True)

In [15]:
#using elastic search
context = "The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations, set up at the request of member governments. It was first established in 1988 by two United Nations organizations, the World Meteorological Organization (WMO) and the United Nations Environment Programme (UNEP), and later endorsed by the United Nations General Assembly through Resolution 43/53. Membership of the IPCC is open to all members of the WMO and UNEP. The IPCC produces reports that support the United Nations Framework Convention on Climate Change (UNFCCC), which is the main international treaty on climate change. The ultimate objective of the UNFCCC is to \"stabilize greenhouse gas concentrations in the atmosphere at a level that would prevent dangerous anthropogenic [i.e., human-induced] interference with the climate system\". IPCC reports cover \"the scientific, technical and socio-economic information relevant to understanding the scientific basis of risk of human-induced climate change, its potential impacts and options for adaptation and mitigation.\""

qa({
    'question': 'What organization is the IPCC a part of?',
    'context': context
})

{'score': 0.48815762996673584,
 'start': 118,
 'end': 133,
 'answer': 'United Nations,'}

In [4]:
import json

with open('squad/dev1.json', 'r') as f:
    squad = json.load(f)

In [20]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host='localhost', username='root', password='', index='squad_docs')

WARNING - elasticsearch -  HEAD http://localhost:9200/_alias/squad_docs [status:N/A request:4.018s]
Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 245, in perform_request
    response = self.pool.urlopen(
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 755, in u

WARNING - elasticsearch -  HEAD http://localhost:9200/_alias/squad_docs [status:N/A request:4.027s]
Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 245, in perform_request
    response = self.pool.urlopen(
  File "C:\Users\siddh\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 755, in u

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x000001F67DC6E9A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000001F67DC6E9A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it)

In [17]:
path=r'C:\Users\siddh\Desktop\pythonudmy\nlp\model'

In [18]:
from transformers import BertTokenizer, BertForQuestionAnswering
modelname='deepset/bert-base-cased-squad2'
tokenizer=BertTokenizer.from_pretrained(path)
model = BertForQuestionAnswering.from_pretrained(path)

In [19]:
from transformers import pipeline
qa = pipeline('question-answering',model=model,tokenizer=tokenizer)

In [20]:
context = "The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations, set up at the request of member governments."

qa({
    'question': 'What organization is the IPCC a part of?',
    'context': context
})

{'score': 0.599351167678833,
 'start': 114,
 'end': 133,
 'answer': 'the United Nations,'}